In [5]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [6]:
consumer_key = "iZn1gECyAXTEf4tlotlTNVSSP"
consumer_secret = "p58dEB9isznp8jGOBwvOYG9G6TDl0MPFZBzvvqHmQMhGEHo7rr"
access_token = "993089461729849345-OPRXctoMPaSL3GWG1e61tUU9WPjypOo"
access_token_secret = "cvUKaGv4ZfooIlF15ZASUtkNS5wsc3NjW8B7f5QBfejSY"

In [7]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [37]:
query = "Donald Trump loss election"

In [38]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x00000269D4576788>, _json={'created_at': 'Thu Nov 19 16:16:50 +0000 2020', 'id': 1329458621734936576, 'id_str': '1329458621734936576', 'full_text': '@realDonaldTrump @BrianKempGA Trump, stewing over election loss, silent as virus surges @APNews\nhttps://t.co/j1mFs3uFsX', 'truncated': False, 'display_text_range': [30, 119], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}, {'screen_name': 'BrianKempGA', 'name': 'Brian Kemp', 'id': 47437206, 'id_str': '47437206', 'indices': [17, 29]}, {'screen_name': 'APNews', 'name': 'AP News', 'id': 16744503, 'id_str': '16744503', 'indices': [88, 95]}], 'urls': [{'url': 'https://t.co/j1mFs3uFsX', 'expanded_url': 'https://apnews.com/article/election-2020-joe-biden-donald-trump-pandemics-public-health-f0f12ecc43fdd2df652b90b7a3e1b5d4', 'display_url': 'apnews.com/article/electi…', 'indi

In [10]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [63]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [64]:
stream(data=['Donald Trump loss election'] , file_name='new_tweets')

In [65]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Lady Witch,32451,520,"Cincinnati, OH",False,0,3,2020-11-19 16:57:38,RT Ninja Since Donald Trump s decisive loss in...,Neutral
1,Agreed. He's not a four year old and the U.S. ...,Akwasi Owusu-Bempah,4865,5768,Toronto,False,1,0,2020-11-19 16:50:44,RT Donald Trump s infantile refusal to accept ...,Negative
2,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Ilanna Phillips,191014,767,,False,0,3,2020-11-19 16:40:28,RT Donald Trump has increasingly receded from ...,Negative
3,RT @UR_Ninja: Since Donald Trump’s decisive lo...,Fe,58537,109,Nowhere,False,0,60,2020-11-19 16:33:29,RT No one stole the election from Donald Trump...,Neutral
4,RT @MikeOkuda: Donald Trump’s infantile refusa...,Tristan Raven,14014,97,,False,0,1,2020-11-19 16:32:45,Trump stewing over election loss silent as vir...,Neutral


In [66]:
from textblob import TextBlob

In [67]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [68]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [69]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Lady Witch,32451,520,"Cincinnati, OH",False,0,3,2020-11-19 16:57:38,RT No one stole the election from Donald Trump...,Neutral
1,Agreed. He's not a four year old and the U.S. ...,Akwasi Owusu-Bempah,4865,5768,Toronto,False,1,0,2020-11-19 16:50:44,Agreed He s not a four year old and the U S is...,Negative
2,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Ilanna Phillips,191014,767,,False,0,3,2020-11-19 16:40:28,RT No one stole the election from Donald Trump...,Negative
3,RT @UR_Ninja: Since Donald Trump’s decisive lo...,Fe,58537,109,Nowhere,False,0,60,2020-11-19 16:33:29,RT Ninja Since Donald Trump s decisive loss in...,Neutral
4,RT @MikeOkuda: Donald Trump’s infantile refusa...,Tristan Raven,14014,97,,False,0,1,2020-11-19 16:32:45,RT Donald Trump s infantile refusal to accept ...,Neutral


In [70]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Lady Witch,32451,520,"Cincinnati, OH",False,0,3,2020-11-19 16:57:38,RT No one stole the election from Donald Trump...,Positive
1,Agreed. He's not a four year old and the U.S. ...,Akwasi Owusu-Bempah,4865,5768,Toronto,False,1,0,2020-11-19 16:50:44,Agreed He s not a four year old and the U S is...,Positive
2,RT @glenn76833403: @FrankAmari2 @realDonaldTru...,Ilanna Phillips,191014,767,,False,0,3,2020-11-19 16:40:28,RT No one stole the election from Donald Trump...,Positive
3,RT @UR_Ninja: Since Donald Trump’s decisive lo...,Fe,58537,109,Nowhere,False,0,60,2020-11-19 16:33:29,RT Ninja Since Donald Trump s decisive loss in...,Neutral
4,RT @MikeOkuda: Donald Trump’s infantile refusa...,Tristan Raven,14014,97,,False,0,1,2020-11-19 16:32:45,RT Donald Trump s infantile refusal to accept ...,Negative


In [71]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Agreed. He's not a four year old and the U.S. is not a sandbox: Donald Trump’s response to his election loss may be… https://t.co/s0m2sR0DXV

Clean tweet:
 Agreed He s not a four year old and the U S is not a sandbox Donald Trump s response to his election loss may be

Sentiment of the tweet:
 Positive


In [72]:
n = 120
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @molly_knight: so it appears donald trump's election loss plan is the same as his covid-19 battle plan. just deny it's happening and hop…

Clean tweet:
 RT knight so it appears donald trump s election loss plan is the same as his covid 19 battle plan just deny it s happening and hop

Sentiment of the tweet:
 Neutral


In [73]:
df[df.Sentiment == 'Positive'].shape[0]

777

In [74]:
df[df.Sentiment == 'Neutral'].shape[0]

149

In [76]:
df[df.Sentiment == 'Negative'].shape[0]

74